In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import re
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plotting charts
from nltk.corpus import stopwords
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score
from keras.models import Sequential
from keras import layers
from keras.layers import Dense ,LSTM ,Dropout ,BatchNormalization,Bidirectional ,Embedding
import string
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/train.csv")
test_data = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/test.csv")
submission = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/sample_submission.csv")
train_data.head()

In [ ]:
#####################################
##########Perform some Cleaning######
#####################################

nltk.download("stopwords")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()

def word_lemmatize(text):
    text = text.split()
    text = [lemmatizer.lemmatize(w) for w in text]
    text = ' '.join(text)
    return text

train_data['discourse_text'] = train_data['discourse_text'].apply(word_lemmatize)

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train_data['discourse_text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
X = tokenizer.texts_to_sequences(train_data['discourse_text'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)
Z = tokenizer.texts_to_sequences(test_data['discourse_text'].values)
Z = pad_sequences(Z, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', Z.shape)
Y = pd.get_dummies(train_data['discourse_effectiveness']).values
print('Shape of label tensor:', Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.1, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(100,return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(100,return_sequences=False))
model.add(Dropout(0.4))
model.add(Dense(3, activation='sigmoid'))
optimizer = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
epochs = 10
batch_size = 32
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
results = (model.predict(Z))

In [ ]:
df = pd.DataFrame(results)
df = df.rename(columns={0:"Adequate",1:"Ineffective",2:"Effective"})
df_index = pd.DataFrame(test_data.discourse_id)
submission = pd.concat([df_index,df],axis=1)
submission.to_csv('submission.csv', index=False)